# Swiss Inflation Forecast Evaluation

This notebook is dedicated to the exploratory data analysis (EDA) and forecast evaluation for Swiss (CH) headline inflation. It visualizes various model forecasts, evaluates their performance using standard metrics, and generates publication-ready tables summarizing the results.

### Key Objectives:
1.  **Visualize Inflation Data**: Plot the historical Swiss CPI inflation to understand its characteristics.
2.  **Plot Forecasts**: Generate and display fan charts for a suite of forecasting models, including:
    - Baseline models (Naïve, Rolling Mean, PNC)
    - ARIMA models (Auto-ARIMA, Fixed ARIMA)
    - BVAR models (Diffuse, Minnesota, Normal-Wishart priors)
    - Dynamic Factor Model (DFM)
    - Distributional Random Forest (DRF)
3.  **Evaluate Performance**: Compute and display key forecast accuracy metrics (RMSE, MAE, CRPS) for each model across different forecast horizons.
4.  **Summarize Results**: Aggregate the evaluation metrics into a final summary table and save it to a CSV file.
5.  **Generate LaTeX Output**: Create a publication-quality LaTeX table from the summary results.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import sys
from pathlib import Path

# Define the root directory of the project
ROOT_DIR = Path.cwd().parent
sys.path.append(str(ROOT_DIR))

# Define output directories
RESULTS_DIR = ROOT_DIR / "results"
TABLES_DIR = RESULTS_DIR / "tables"

# Create directories if they don't exist
TABLES_DIR.mkdir(parents=True, exist_ok=True)

from helpers import calculate_mae, calculate_rmse, evaluate_forecasts, create_results_table, ForecastFanChart

### Load and preview the Swiss monetary data
Let's load the dataset and take a first look at its structure and contents.

In [ ]:
# Load the data from the processed data directory
df = pd.read_csv(ROOT_DIR / 'data' / 'processed' / 'ch_data_final.csv')

# Show dimensions and first few rows
print("Shape:", df.shape)
print(df.head())

### Basic information and summary statistics
Get a quick overview of the columns, data types, and summary statistics.

In [ ]:
# Convert date to datetime for plotting
df['date'] = pd.to_datetime(df['date'])

# Extract the 'cpi_total_yoy' series and drop NaN values
cpi_total_yoy = df[['date', 'cpi_total_yoy']].dropna()

# Calculate the 20%, 40%, 60%, and 80% splits
length = len(cpi_total_yoy)
splits = [int(length * p) for p in [0.2, 0.4, 0.6, 0.8]]

# Extract the corresponding dates
split_dates = [cpi_total_yoy.iloc[split]['date'] for split in splits]

# Plot the inflation series
plt.figure(figsize=(15, 7))
plt.plot(df['date'], df['cpi_total_yoy'], linewidth=2, color='black', label='YoY CPI Inflation')
plt.axhline(0, linestyle='--', color='grey', linewidth=0.8)

# Add vertical lines for the split dates
for split_date in split_dates:
    plt.axvline(x=split_date, color='red', linestyle='--', linewidth=1, label=f'{split_date.year}')

plt.title('Swiss CPI Inflation (YoY %)', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Inflation Rate (%)')
plt.grid(True, axis='y', linestyle='--', linewidth=0.5)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Setup
# Define the start date for cropping the time series
start_date = split_dates[1]

# Create the actuals series needed for the fan chart's background plot
pi_series = pd.Series(
    df.loc[df['date'] >= pd.to_datetime(start_date), 'cpi_total_yoy'].values,
    index=pd.to_datetime(df.loc[df['date'] >= pd.to_datetime(start_date), 'date'])
)

# Instantiate the fan chart plotter with the actuals
fan_chart = ForecastFanChart(pi_series)

# Initialize empty list to collect all metrics
all_metrics_list = []

In [ ]:
# Helper function to process model forecasts
def process_model(model_name, path, fan_chart_plotter, start_date, color, method="normal"):
    """
    Plots fan chart, evaluates forecast, and returns metrics.
    Handles both .pkl and .csv files.
    """
    if str(path).endswith(".pkl"):
        forecast_df = pd.read_pickle(path)
        fan_chart_plotter.plot_from_python(
            path,
            horizon_step=1,
            title=f"Swiss CPI Inflation: {model_name} Forecast (1-Month Horizon)",
            start_date=start_date,
            color=color,
            method=method
        )
    elif str(path).endswith(".csv"):
        forecast_df = pd.read_csv(path)
        if 'last_20_values' in forecast_df.columns:
            # Convert string representation of list to actual list of floats
            forecast_df['last_20_values'] = forecast_df['last_20_values'].apply(
                lambda x: [float(i.strip()) for i in x.strip('[]').split(',')]
            )
        
        # The plot_from_python function now handles CSVs as well
        fan_chart_plotter.plot_from_python(
            path,
            horizon_step=1,
            title=f"Swiss CPI Inflation: {model_name} Forecast (1-Month Horizon)",
            start_date=start_date,
            color=color,
            method=method
        )
    else:
        raise ValueError(f"Unsupported file type for {path}")

    metrics = evaluate_forecasts(forecast_df, method=method, start_date=start_date)
    metrics['Model'] = model_name
    
    print(f"--- {model_name} Metrics ---")
    print(metrics)
    
    return metrics

# --- 1. Process Baseline and BVAR Models ---
model_definitions = {
    "Naive Mean": ("baseline/ch_naive_mean.csv", "red", "normal"),
    "Naive Last": ("baseline/ch_naive_last.csv", "blue", "normal"),
    "PNC": ("baseline/ch_pnc_model.csv", "green", "empirical"),
    "BVAR Diffuse": ("bvar/ch_bvar_diffuse.csv", "purple", "normal"),
    "BVAR Minnesota": ("bvar/ch_bvar_minnesota.csv", "orange", "normal"),
    "BVAR Normal-Wishart": ("bvar/ch_bvar_normalwishart.csv", "brown", "normal"),
    "ARIMA(1,1,0)": ("baseline/ch_arima_110.csv", "blue", "normal"),
    "DFM": ("dfm/ch_dfm.csv", "red", "normal")
}

for name, (path, color, method) in model_definitions.items():
    full_path = ROOT_DIR / "results" / "forecasts" / path
    metrics = process_model(name, full_path, fan_chart, start_date, color, method)
    all_metrics_list.append(metrics)

In [ ]:
# --- DRF ---
ch_drf_path = ROOT_DIR / "results" / "forecasts" / "drf" / "ch_drf_forecast_results.csv"

# Plot the 1-month horizon fan chart from the CSV
fan_chart.plot_from_r(
    ch_drf_path,
    horizon_step=1, # Specify the horizon to plot
    title="Swiss CPI Inflation: DRF Forecast (1-Month Horizon)",
    start_date=start_date,
    color="purple"
)

# Load the evaluation metrics from the R-generated CSV
r_metrics_path = ROOT_DIR / "results" / "forecasts" / "drf" / "drf_evaluation_summary.csv"
all_r_metrics_df = pd.read_csv(r_metrics_path)
ch_drf_metrics = all_r_metrics_df[all_r_metrics_df['Model'] == 'DRF_CH'].copy()
ch_drf_metrics['Model'] = 'DRF'
all_metrics_list.append(ch_drf_metrics)
print("--- DRF Metrics ---")
print(ch_drf_metrics)

In [ ]:
# Combine all metrics into a single DataFrame
final_results_df = pd.concat(all_metrics_list, ignore_index=True)

# Reorder columns for better readability
final_results_df = final_results_df[['Model', 'horizon_step', 'RMSE', 'MAE', 'CRPS']]

# Save the combined results to a CSV file
output_path = TABLES_DIR / 'ch_results_summary.csv'
final_results_df.to_csv(output_path, index=False, float_format='%.4f')

# Print the final summary table
print("      COMBINED FORECAST EVALUATION SUMMARY (CH)")
print(final_results_df)
print(f"\nSummary results have been saved to '{output_path}'")

In [ ]:
# --- Generate Publication-Ready LaTeX Table ---

# 1. Define the structure and naming for your table
REGION_NAME = "Switzerland (CH)"  # <-- Define the region name here
HORIZON_GROUPS = [[1, 2], [3, 6], [12]]
MODEL_GROUPS = {
    "Baseline Models": ["Naive Last", "Naive Mean", "PNC", "ARIMA(1,1,0)"],
    "BVAR Models": ["BVAR Diffuse", "BVAR Minnesota", "BVAR Normal-Wishart"],
    "Multivariate \\& ML Models": ["DFM", "DRF"]
}
MODEL_NAME_MAP = {
    "Naive Last": "Naive last",
    "Naive Mean": "Rolling mean",
    "PNC": "PNC",
    "ARIMA(1,1,0)": "ARIMA(1,1,0)",
    "BVAR Diffuse": "BVAR (diffuse)",
    "BVAR Minnesota": "BVAR (Minnesota)",
    "BVAR Normal-Wishart": "BVAR (NW)",
    "DFM": "DFM",
    "DRF": "DRF"
}

# 3. Generate the complete LaTeX code using the new function name and argument
publication_latex_code = create_results_table(
    df=final_results_df,
    region_name=REGION_NAME,
    horizon_groups=HORIZON_GROUPS,
    model_groups=MODEL_GROUPS,
    model_name_map=MODEL_NAME_MAP
)

# 4. Save the code to a dynamic .tex file
short_region_name = REGION_NAME.split('(')[-1].replace(')', '').lower().strip()
latex_output_path = TABLES_DIR / f'{short_region_name}_results_table.tex'
with open(latex_output_path, 'w') as f:
    f.write(publication_latex_code)

# 5. Print the LaTeX code to the console
print(f"\nSaved the following LaTeX code to '{latex_output_path}'.")
print(publication_latex_code)